[View in Colaboratory](https://colab.research.google.com/github/findingfoot/ML_practice-codes/blob/master/Different_layers_in_Neural_Network.ipynb)

Info:
    We will be dealing with different kinds of layers in Neural Networks
    
    1. Convolution Layer
    2. Activation Layer
    3. Max Pool layer
    4. Full connected layer
    
    Kind of datasets used:
        1. 1D data emulating row wise data
        2. 2D data emulating images

In [0]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
import matplotlib.pyplot as plt
import random
import csv
import os


In [0]:
ops.reset_default_graph()

sess = tf.Session()


#conv layer stuff
size_of_data = 25
conv_filter_size = 5
maxpool_size = 5
stride = 1

In [0]:
#reproducibility

seed = 23
np.random.seed(seed)
tf.set_random_seed(seed)

In [0]:
#Begin with 1D data

data_1d = np.random.normal(size=size_of_data)

#now create a placeholder 
x_input_1d = tf.placeholder(dtype=tf.float32, shape=[size_of_data])


In [0]:
#create the convolution

def conv_layer_1d(input_1d, my_filter, stride):
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    
    conv_output = tf.nn.conv2d(input_4d, filter = my_filter, strides = [1,1,stride,1], padding = 'VALID')
    
    #removing the added dimension
    
    conv_output_1d = tf.squeeze(conv_output)
    return(conv_output_1d)

my_filter = tf.Variable(tf.random_normal(shape = [1,conv_filter_size,1,1]))

my_convolution_output = conv_layer_1d(x_input_1d,my_filter, stride)

In [0]:
#activation function
def activation(input_1d):
    return(tf.nn.relu(input_1d))

#creating activation layer
my_activation_output = activation(my_convolution_output)

In [0]:
#max pooling layer

def max_pool(input_1d, width, stride):
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    
    #specify the window size
    
    pool_output = tf.nn.max_pool(input_4d, 
                                 ksize=[1,1,width,1], 
                                 strides= [1,1,stride,1],
                                padding = "VALID")
    
    #removing extra dimension
    pool_output_1d = tf.squeeze(pool_output)
    
    return(pool_output_1d)

my_max_pool_output = max_pool(my_activation_output, width = maxpool_size, stride = stride)

In [0]:
#fully connected layer

def fully_connected(input_layer, num_outputs):
    #shape of multiplication weight matrix
    print('shape of input layer',(num_outputs))
    weight_shape = tf.squeeze(tf.stack([tf.shape(input_layer), [num_outputs]]))
    
    #initialize weight
    weight = tf.random_normal(weight_shape, stddev=0.1)
    
    #initialize the bias
    
    bias = tf.random_normal(shape=[num_outputs])
    
    #converting 1d array into 2d array
    input_layer_2d = tf.expand_dims(input_layer, 0)
    
    #do the matrix multiplication and add bias
    full_output = tf.add(tf.matmul(input_layer_2d, weight), bias)
    
    #squeeze out the extra dimension
    
    full_output_1d = tf.squeeze(full_output)
    
    return (full_output_1d)

my_full_output = fully_connected(my_max_pool_output, 5)
    

shape of input layer 5


In [0]:
#lets run the graph and see

init = tf.global_variables_initializer()
sess.run(init)

feed_dict = {x_input_1d: data_1d}

print('++++++++++++++++ Running 1D data ++++++++++++++++++++++ \n')

print('Input = array of length %d ' %(x_input_1d.shape.as_list()[0]))
print('convolution with filter, length = %d, stride size = %d, results in an array of length %d : \n' 
      % (conv_filter_size, stride, my_convolution_output.shape.as_list()[0]))

print(sess.run(my_convolution_output, feed_dict=feed_dict))


print('\n\n+++++++++++ Activation Block +++++++++++++++++')

print('\n Input = above array of length %d' % (my_convolution_output.shape.as_list()[0]))
print('\n Relu activation function returns an array of length %d' %(my_activation_output.shape.as_list()[0]))
print(sess.run(my_activation_output, feed_dict=feed_dict))


print('++++++++++++++++++++++ Max Pool layer +++++++++++++++++++')
print('\n Input = above array of length %d' % (my_activation_output.shape.as_list()[0]))
print('Max Pool, window length = %d, stride size = %d, results in an array of length %d : \n' 
      % (maxpool_size,stride, my_max_pool_output.shape.as_list()[0]))
print(sess.run(my_max_pool_output, feed_dict=feed_dict))


print('\n++++++++++++++++++++++ Fully Connected Layer ++++++++++++++++++')
print('\n Input = above array of length %d' % (my_max_pool_output.shape.as_list()[0]))
print('Fully connected layer on all 4 rows with %d outputs:' % 
      (my_full_output.shape.as_list()[0]))
print(sess.run(my_full_output, feed_dict=feed_dict))

++++++++++++++++ Running 1D data ++++++++++++++++++++++ 

Input = array of length 25 
convolution with filter, length = 5, stride size = 1, results in an array of length 21 : 

[-0.41701764 -2.7259521   1.4663595   1.9974064   1.0364381   2.8468585
 -2.5038176  -2.2123046   0.54763734 -0.9993551  -0.12350886  1.67453
  2.634409   -0.34433162 -3.4637158   1.6823386   3.9866822   0.10009709
 -2.3700492  -1.909903   -0.8372239 ]


+++++++++++ Activation Block +++++++++++++++++

 Input = above array of length 21

 Relu activation function returns an array of length 21
[0.         0.         1.4663595  1.9974064  1.0364381  2.8468585
 0.         0.         0.54763734 0.         0.         1.67453
 2.634409   0.         0.         1.6823386  3.9866822  0.10009709
 0.         0.         0.        ]
++++++++++++++++++++++ Max Pool layer +++++++++++++++++++

 Input = above array of length 21
Max Pool, window length = 5, stride size = 1, results in an array of length 17 : 

[1.9974064  2.8468585